<a href="https://colab.research.google.com/github/TejaswiAravelli/AI/blob/main/assg5multiplelinearregression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [39]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [40]:
import numpy as np
import math as mt
import random
import pandas as pd


In [3]:
data=pd.read_csv('/content/drive/MyDrive/sample2.csv')
data.head()


,X1,X2,X3,Y
0,5551.82208,4983.17184,4888.39680,5072.95872
1,4983.17184,4888.39680,5072.95872,5196.25980
2,4888.39680,5072.95872,5196.25980,5641.29720
3,5072.95872,5196.25980,5641.29720,7153.17732
4,5196.25980,5641.29720,7153.17732,7849.80504


In [4]:
outliers = []
def detect_outliers_zscore(data):
    thres = 3
    mean = np.mean(data)
    std = np.std(data)
    # print(mean, std)
    for i in data:
        z_score = (i-mean)/std
        if (np.abs(z_score) > thres):
            outliers.append(i)
    return outliers # Driver code


In [5]:
sample_outliers_X1 = detect_outliers_zscore(data['X1'])
sample_outliers_X2 = detect_outliers_zscore(data['X2'])
sample_outliers_X3 = detect_outliers_zscore(data['X3'])
sample_outliers_Y = detect_outliers_zscore(data['Y'])
print("Outliers in X1 column from Z-scores method : ", sample_outliers_X1)
print("Outliers in X2 column from Z-scores method : ", sample_outliers_X2)
print("Outliers in X3 column from Z-scores method : ", sample_outliers_X3)
print("Outliers in Y column from Z-scores method : ", sample_outliers_Y)


Outliers in X1 column from Z-scores method :  []
Outliers in X2 column from Z-scores method :  []
Outliers in X3 column from Z-scores method :  []
Outliers in Y column from Z-scores method :  []


In [6]:
print(data.isnull().sum())

X1    0
X2    1
X3    2
Y     3
dtype: int64


In [41]:
from sklearn.preprocessing import MinMaxScaler
X1 = np.array(data['X1']).reshape(-1,1)


In [8]:
scaler = MinMaxScaler()
scaler.fit(X1)
X1_scaled = scaler.transform(X1)
data['X1'] = X1_scaled.reshape(1,-1)[0]
X2 = np.array(data['X2']).reshape(-1,1)
scaler = MinMaxScaler()
scaler.fit(X2)
X2_scaled = scaler.transform(X2)
data['X2'] = X2_scaled.reshape(1,-1)[0]
X3 = np.array(data['X3']).reshape(-1,1)
scaler = MinMaxScaler()
scaler.fit(X3)
X3_scaled = scaler.transform(X3)
data['X3'] = X3_scaled.reshape(1,-1)[0]
Y1 = np.array(data['Y']).reshape(-1,1)
scaler = MinMaxScaler()
scaler.fit(Y1)
Y_scaled = scaler.transform(Y1)
data['Y'] = Y_scaled.reshape(1,-1)[0]


In [9]:
m1=data['X1'].mean()
m2=data['X2'].mean()
m3=data['X3'].mean()
m4=data['Y'].mean()
data['X1'].fillna(value=m1, inplace=True)
data['X2'].fillna(value=m2, inplace=True)
data['X3'].fillna(value=m3, inplace=True)
data['Y'].fillna(value=m4, inplace=True)
data


,X1,X2,X3,Y
0,0.397877,0.293800,0.276454,0.310234
1,0.293800,0.276454,0.310234,0.332801
2,0.276454,0.310234,0.332801,0.414254
3,0.310234,0.332801,0.414254,0.690965
4,0.332801,0.414254,0.690965,0.818465
...,...,...,...,...
2179,0.228524,0.217797,0.246298,0.324384
2180,0.217797,0.246298,0.324384,0.651650
2181,0.246298,0.324384,0.651650,0.485276
2182,0.324384,0.651650,0.485180,0.485276


In [10]:
from sklearn.model_selection import train_test_split
dataframe = data.values
X, y = dataframe[:, :-1], dataframe[:, -1]
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 10)
x_train


array([[0.45539357, 0.40426807, 0.2812188 ],
       [0.73977918, 0.77492796, 0.74297452],
       [0.53447833, 0.48064249, 0.41097258],
       ...,
       [0.61011098, 0.44563636, 0.4770192 ],
       [0.22641294, 0.17160709, 0.20062195],
       [0.4363071 , 0.41339762, 0.37793501]])

In [23]:
m1=random.uniform(-10,10)
m2=random.uniform(-10,10)
m3=random.uniform(-10,10)
c=random.uniform(-10,10)
eta=0.01
max_iter=1000


In [24]:
def gradient(m1,m2,m3,c,x1,x2,x3,y):
  m1_deriv=-1*(y-m1*x1-m2*x2-m3*x3-c)*x1
  m2_deriv=-1*(y-m1*x1-m2*x2-m3*x3-c)*x2
  m3_deriv=-1*(y-m1*x1-m2*x2-m3*x3-c)*x3
  c_deriv=-1*(y-m1*x1-m2*x2-m3*x3-c)
  return m1_deriv,m2_deriv,m3_deriv,c_deriv


In [25]:
for iter in range(max_iter):
  for sample in range(x_train.shape[0]):
    del_m1,del_m2,del_m3,del_c=gradient(m1,m2,m3,c,x_train[sample,0],x_train[sample,1],x_train[sample,2],y_train[sample])
    m1=m1-eta*del_m1
    m2=m2-eta*del_m2
    m3=m3-eta*del_m3
    c=c-eta*del_c
print('Model parameters m1, m2, m3 and c : ',m1,m2,c)


Model parameters m1, m2, m3 and c :  -0.2124311540356671 0.03716381751165487 0.1521226651306219


In [26]:
y_pred_train = []
for sample in range(x_train.shape[0]):
  x1=x_train[sample,0]
  x2=x_train[sample,1]
  x3=x_train[sample,2] 
  y=y_train[sample]
  y_p_tr = (m1 * x1) + (m2 * x2) + (m3 * x3) + c
  y_pred_train.append(y_p_tr)
y_pred_train = np.array(y_pred_train)


In [33]:
y_pred_test = []
for sample in range(x_test.shape[0]):
  x1=x_test[sample,0]
  x2=x_test[sample,1]
  x3=x_test[sample,2] 
  y=y_test[sample]
  y_p_te = (m1 * x1) + (m2 * x2) + (m3 * x3) + c
  y_pred_test.append(y_p_te)
y_pred_test = np.array(y_pred_test)


In [34]:
import math
from sklearn.metrics import mean_squared_error 
from sklearn.metrics import mean_absolute_error


In [35]:
#Training Accuracies
mse = mean_squared_error(y_train, y_pred_train) 
print('Mean square error ', mse)
rmse = math.sqrt(mean_squared_error(y_train, y_pred_train)) 
print('Root mean square error ', rmse)  
mae=mean_absolute_error(y_train, y_pred_train)
print('Mean absolute error ', mae)


Mean square error  0.013175708078244502
Root mean square error  0.11478548722832735
Mean absolute error  0.08026615932368818


In [42]:
#Testing Accuracies
mse = mean_squared_error(y_test, y_pred_test)
print('Mean square error ', mse)
rmse = math.sqrt(mean_squared_error(y_test, y_pred_test)) 
print('Root mean square error ', rmse)  
mae=mean_absolute_error(y_test, y_pred_test)
print('Mean absolute error ', mae)


Mean square error  0.015734017372649758
Root mean square error  0.1254353115061694
Mean absolute error  0.08586544367593865


In [43]:
X1=input('Enter Load at T1: ')
X2=input('Enter Load at T2: ')
X3=input('Enter Load at T3: ')
Y=m1*float(X1)+m2*float(X2)+m3*float(X3)+c
print(' Load at Time T : ',Y)


Enter Load at T1: 4340
Enter Load at T2: 5430
Enter Load at T3: 3000
 Load at Time T :  1872.6712268788135
